<a href="https://colab.research.google.com/github/la2015-hw/Group_10/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"janyarathnakumar","key":"d42a537249f1d7ddac059650d1758fef"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets list

ref                                                            title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
ahmeduzaki/global-earthquake-tsunami-risk-assessment-dataset   Global Earthquake-Tsunami Risk Assessment Dataset       16151  2025-10-01 16:35:53.273000          15991        541  1.0              
jockeroika/life-style-data                                     Life Style Data                                       3995645  2025-10-14 13:50:45.303000          18708        378  0.8235294        
jaderz/hospital-beds-management                                Hospital Beds Management                                47583  2025-10-03 09:21:58.590000          13009        326  1.0              
ayeshaimra

In [4]:
!kaggle datasets download -d vaishnavichintha/sdss-tabular-dataset

Dataset URL: https://www.kaggle.com/datasets/vaishnavichintha/sdss-tabular-dataset
License(s): unknown
  0% 0.00/10.7M [00:00<?, ?B/s]
100% 10.7M/10.7M [00:00<00:00, 1.09GB/s]


In [5]:
!unzip sdss-tabular-dataset.zip

Archive:  sdss-tabular-dataset.zip
  inflating: sdss_gz2_matched_cleaned_balanced.csv  


In [6]:
import pandas as pd
df = pd.read_csv("sdss_gz2_matched_cleaned_balanced.csv")
print(df.head())

             specobjid      dr8objid            dr7objid          ra  \
0   614788390251948032  1.237654e+18  587728665578700971  122.952644   
1  2433194008562919424  1.237665e+18  587739651567845579  231.428600   
2  2080728518915811328  1.237663e+18  587736975809773641  232.551060   
3  1563980851195176960  1.237662e+18  587736619321917717  234.050800   
4  3139079110530197504  1.237666e+18  587739843772940458  217.789000   

         dec     rastring    decstring    sample gz2class  \
0  42.216805  08:11:48.63  +42:13:00.5  original    SBc?m   
1  23.370867  15:25:42.86  +23:22:15.1  original      Ser   
2  27.134108  15:30:12.25  +27:08:02.8  original    SBc2m   
3  30.967339  15:36:12.19  +30:58:02.4  original    SBd2m   
4  20.449600  14:31:09.36  +20:26:58.6  original    SBa2l   

   total_classifications  ...  \
0                     47  ...   
1                     40  ...   
2                     44  ...   
3                     55  ...   
4                     34  ...   

 

In [7]:
x = df.drop(columns=["broad_class"])  # all features
y = df["broad_class"]                 # target (galaxy morphology)


In [8]:
df.select_dtypes(include=['object']).columns


Index(['rastring', 'decstring', 'sample', 'gz2class', 'broad_class'], dtype='object')

In [9]:
# Drop non-numeric or target columns
x = df.drop(columns=[
    "sample",
    "broad_class",    # target column
    "rastring", "decstring",      # coordinates
    "gz2class"        # text morphological shorthand
])

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, df["broad_class"], test_size=0.3, random_state=25, stratify=df["broad_class"]
)

print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)


Train shape: (33320, 229) (33320,)
Test shape: (14280, 229) (14280,)


In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

dt = DecisionTreeClassifier(criterion="gini", random_state=25)
dt.fit(x_train, y_train)

y_pred = dt.predict(x_test)

print(f"Decision Tree Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Decision Tree Test Accuracy: 0.9940

Classification Report:
                 precision    recall  f1-score   support

     Elliptical       0.99      0.99      0.99      3570
      Irregular       0.99      1.00      1.00      3570
  Spiral-barred       1.00      1.00      1.00      3570
Spiral-unbarred       0.99      0.99      0.99      3570

       accuracy                           0.99     14280
      macro avg       0.99      0.99      0.99     14280
   weighted avg       0.99      0.99      0.99     14280


Confusion Matrix:
[[3529   13    5   23]
 [   7 3560    1    2]
 [   8    3 3555    4]
 [  16    2    1 3551]]
